<a href="https://colab.research.google.com/github/redditgoogle/EIP3_2_Assignment2/blob/master/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [4]:
from google.colab import files
files.upload()

Saving Alice.txt to Alice (1).txt


{'Alice.txt': b'\xef\xbb\xbfProject Gutenberg\'s Alice\'s Adventures in Wonderland, by Lewis Carroll\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: Alice\'s Adventures in Wonderland\r\n\r\nAuthor: Lewis Carroll\r\n\r\nPosting Date: June 25, 2008 [EBook #11]\r\nRelease Date: March, 1994\r\n[Last updated: December 20, 2011]\r\n\r\nLanguage: English\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nALICE\'S ADVENTURES IN WONDERLAND\r\n\r\nLewis Carroll\r\n\r\nTHE MILLENNIUM FULCRUM EDITION 3.0\r\n\r\n\r\n\r\n\r\nCHAPTER I. Down the Rabbit-Hole\r\n\r\nAlice was beginning to get very tired of sitting by her sister on the\r\nbank, and of having nothing to do: once or twice she had peeped into the\r\nbook her sister was reading, but it had no pictures or c

In [3]:
import string
# load ascii text and covert to lowercase
filename = "Alice.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
raw_text.translate(str.maketrans('', '', string.punctuation))

'\ufeffproject gutenbergs alices adventures in wonderland by lewis carroll\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever  you may copy it give it away or\nreuse it under the terms of the project gutenberg license included\nwith this ebook or online at wwwgutenbergorg\n\n\ntitle alices adventures in wonderland\n\nauthor lewis carroll\n\nposting date june 25 2008 ebook 11\nrelease date march 1994\nlast updated december 20 2011\n\nlanguage english\n\n\n\n\n\n\n\n\n\n\n\n\n\nalices adventures in wonderland\n\nlewis carroll\n\nthe millennium fulcrum edition 30\n\n\n\n\nchapter i down the rabbithole\n\nalice was beginning to get very tired of sitting by her sister on the\nbank and of having nothing to do once or twice she had peeped into the\nbook her sister was reading but it had no pictures or conversations in\nit and what is the use of a book thought alice without pictures or\nconversations\n\nso she was considering in her own mind 

In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [5]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163703
Total Vocab:  59


In [6]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 500
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163203


In [7]:
from keras.preprocessing.sequence import pad_sequences
# define sequences
sequences = [
	[1, 2, 3, 4],
	   [1, 2, 3],
		     [1]
	]
# pad sequence
padded = pad_sequences(sequences)
print(padded)

[[1 2 3 4]
 [0 1 2 3]
 [0 0 0 1]]


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [9]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0723 18:02:26.692085 140193764923264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0723 18:02:26.732091 140193764923264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 18:02:26.737090 140193764923264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0723 18:02:27.121831 140193764923264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0723 18:02:27.136106 

In [0]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
model.fit(X, y, epochs=100, batch_size=512, callbacks=callbacks_list)

W0723 18:02:27.372319 140193764923264 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
163203/163203 [==============================] - 364s 2ms/step - loss: 3.0670

Epoch 00001: loss improved from inf to 3.06699, saving model to weights-improvement-01-3.0670.hdf5
Epoch 2/100
163203/163203 [==============================] - 362s 2ms/step - loss: 2.9262

Epoch 00002: loss improved from 3.06699 to 2.92622, saving model to weights-improvement-02-2.9262.hdf5
Epoch 3/100
163203/163203 [==============================] - 361s 2ms/step - loss: 2.8564

Epoch 00003: loss improved from 2.92622 to 2.85638, saving model to weights-improvement-03-2.8564.hdf5
Epoch 4/100
 10240/163203 [>.............................] - ETA: 5:37 - loss: 2.8303